# CNN (Convolutional Neural Network)

### 1, Padding and Stride

$$
OH = \frac{W + 2P - FH}{S} + 1 \\
OW = \frac{W + 2P - FW}{S} + 1
$$

Example
- input size: (4, 4) padding: 1, stride: 1 filter size(3, 3)
$$
OH = \frac{4 + 2*1 -3}{1} + 1 = 4 \\
OW = \frac{4 + 2*1 - 3}{1} + 1 = 4
$$

- input size: (7, 7) padding: 0 stride: 2, filter size: (3, 3)

$$
OH = \frac{7 + 2*0 -3}{2} + 1 = 3 \\
OW = \frac{7 + 2*0 -3}{2} + 1 = 3
$$

- input size: (28, 31), padding: 2, stride: 3, filter size: (5, 5)

$$
OH = \frac{28 + 2*2 - 5}{3} + 1 = 10 \\
OW = \frac{31 + 2*2 - 5}{3} + 1 = 11
$$

In [3]:
import numpy as np
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [5]:
x[0].shape
x[1].shape

(1, 28, 28)

In [ ]:
x[0, 0]

### 2, Execute convolution layer

In [1]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from shared_code.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)
